<a href="https://colab.research.google.com/github/CS20M038/CS6910-assignment_3/blob/main/Attebtion_with_wandB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Start Creating the Dataset

In [1]:
#@title Import Libraries
from random import randint
from numpy import array
from numpy import argmax
import keras.backend as K
from tensorflow.keras import models
from numpy import array_equal
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Lambda
from tensorflow.keras import backend as K
from tensorflow.compat.v1.keras.layers import CuDNNGRU
from scipy.ndimage.interpolation import shift
import csv
import random
!pip install wandb
import wandb

     |████████████████████████████████| 1.8MB 28.4MB/s 
     |████████████████████████████████| 102kB 14.8MB/s 
     |████████████████████████████████| 174kB 53.7MB/s 
     |████████████████████████████████| 133kB 57.5MB/s 
     |████████████████████████████████| 71kB 12.3MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=be4c126307af7d47ea000fb1d8ab2037424102e082fa0f75edbc1da902169153
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=2246c5ed437ba6ac28f0ebb4b9d1b383e7c15389e2984fd17ea18a94a5fe6e3a
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [2]:
#@title Check GPU
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
  print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
input_dict=['<start>','<end>']
target_dict=['<start>','<end>']
eng_alphabets = 'abcdefghijklmnopqrstuvwxyz'
pad_char = '<end>'
eng_alpha2index = {'<start>': 0,'<end>':1}
for index, alpha in enumerate(eng_alphabets):
    eng_alpha2index[alpha] = index+2
    input_dict.append(alpha)

print(eng_alpha2index)
print(input_dict)
# Hindi Unicode Hex Range is 2304:2432. Source: https://en.wikipedia.org/wiki/Devanagari_(Unicode_block)

hindi_alphabets = [chr(alpha) for alpha in range(2304, 2432)]
hindi_alphabet_size = len(hindi_alphabets)

hindi_alpha2index = {'<start>': 0,pad_char: 1}
#hindi_alpha2index = {'<start>': 0}
for index, alpha in enumerate(hindi_alphabets):
    hindi_alpha2index[alpha] = index+2
    target_dict.append(alpha)
    #print(alpha)

print(hindi_alpha2index)
print(target_dict)



{'<start>': 0, '<end>': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27}
['<start>', '<end>', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
{'<start>': 0, '<end>': 1, 'ऀ': 2, 'ँ': 3, 'ं': 4, 'ः': 5, 'ऄ': 6, 'अ': 7, 'आ': 8, 'इ': 9, 'ई': 10, 'उ': 11, 'ऊ': 12, 'ऋ': 13, 'ऌ': 14, 'ऍ': 15, 'ऎ': 16, 'ए': 17, 'ऐ': 18, 'ऑ': 19, 'ऒ': 20, 'ओ': 21, 'औ': 22, 'क': 23, 'ख': 24, 'ग': 25, 'घ': 26, 'ङ': 27, 'च': 28, 'छ': 29, 'ज': 30, 'झ': 31, 'ञ': 32, 'ट': 33, 'ठ': 34, 'ड': 35, 'ढ': 36, 'ण': 37, 'त': 38, 'थ': 39, 'द': 40, 'ध': 41, 'न': 42, 'ऩ': 43, 'प': 44, 'फ': 45, 'ब': 46, 'भ': 47, 'म': 48, 'य': 49, 'र': 50, 'ऱ': 51, 'ल': 52, 'ळ': 53, 'ऴ': 54, 'व': 55, 'श': 56, 'ष': 57, 'स': 58, 'ह': 59, 'ऺ': 60, 'ऻ': 61, '़': 62, 'ऽ': 63, 'ा': 64, 'ि': 65, '

In [4]:
# Hindi Unicode Hex Range is 2304:2432. Source: https://en.wikipedia.org/wiki/Devanagari_(Unicode_block)

hindi_alphabets = [chr(alpha) for alpha in range(2304, 2432)]
hindi_alphabet_size = len(hindi_alphabets)

hindi_index2alpha = {0:'<start>',1: pad_char}
for index, alpha in enumerate(hindi_alphabets):
    hindi_index2alpha[index+2] = alpha
   # target_dict.append(alpha)
    #print(alpha)

print(hindi_index2alpha)

eng_alphabets = 'abcdefghijklmnopqrstuvwxyz'
eng_index2alpha = {0:'<start>',1: pad_char}
for index, alpha in enumerate(eng_alphabets):
    eng_index2alpha[index+2] = alpha
   # target_dict.append(alpha)
    #print(alpha)

print(eng_index2alpha)

{0: '<start>', 1: '<end>', 2: 'ऀ', 3: 'ँ', 4: 'ं', 5: 'ः', 6: 'ऄ', 7: 'अ', 8: 'आ', 9: 'इ', 10: 'ई', 11: 'उ', 12: 'ऊ', 13: 'ऋ', 14: 'ऌ', 15: 'ऍ', 16: 'ऎ', 17: 'ए', 18: 'ऐ', 19: 'ऑ', 20: 'ऒ', 21: 'ओ', 22: 'औ', 23: 'क', 24: 'ख', 25: 'ग', 26: 'घ', 27: 'ङ', 28: 'च', 29: 'छ', 30: 'ज', 31: 'झ', 32: 'ञ', 33: 'ट', 34: 'ठ', 35: 'ड', 36: 'ढ', 37: 'ण', 38: 'त', 39: 'थ', 40: 'द', 41: 'ध', 42: 'न', 43: 'ऩ', 44: 'प', 45: 'फ', 46: 'ब', 47: 'भ', 48: 'म', 49: 'य', 50: 'र', 51: 'ऱ', 52: 'ल', 53: 'ळ', 54: 'ऴ', 55: 'व', 56: 'श', 57: 'ष', 58: 'स', 59: 'ह', 60: 'ऺ', 61: 'ऻ', 62: '़', 63: 'ऽ', 64: 'ा', 65: 'ि', 66: 'ी', 67: 'ु', 68: 'ू', 69: 'ृ', 70: 'ॄ', 71: 'ॅ', 72: 'ॆ', 73: 'े', 74: 'ै', 75: 'ॉ', 76: 'ॊ', 77: 'ो', 78: 'ौ', 79: '्', 80: 'ॎ', 81: 'ॏ', 82: 'ॐ', 83: '॑', 84: '॒', 85: '॓', 86: '॔', 87: 'ॕ', 88: 'ॖ', 89: 'ॗ', 90: 'क़', 91: 'ख़', 92: 'ग़', 93: 'ज़', 94: 'ड़', 95: 'ढ़', 96: 'फ़', 97: 'य़', 98: 'ॠ', 99: 'ॡ', 100: 'ॢ', 101: 'ॣ', 102: '।', 103: '॥', 104: '०', 105: '१', 106: '२', 107: '३', 108: '४', 109: '५',

In [5]:
def readXmlDataset(filename):
        tsv_file = open(filename)
        read_tsv = csv.reader(tsv_file, delimiter="\t")
        lang1_words = []
        lang2_words = []

        for row in read_tsv:
            lang2_words.append(row[0])
            lang1_words.append(row[1])
            #print(row[0])

        return lang1_words, lang2_words
train_input_texts, train_target_texts = readXmlDataset('hi.translit.sampled.train.tsv')
test_input_texts, test_target_texts = readXmlDataset('hi.translit.sampled.test.tsv')
val_input_texts, val_target_texts = readXmlDataset('hi.translit.sampled.dev.tsv')

In [6]:
train_input_texts[0]

'an'

In [7]:
def word_rep(word, letter2index, device = 'cpu'):
    rep = np.zeros((len(word)+1,  129))
    for letter_index, letter in enumerate(word):
        pos = letter2index[letter]
        rep[letter_index][pos] = 1
    pad_pos = letter2index[pad_char]
    rep[letter_index+1][pad_pos] = 1
    return rep

def gt_rep(word, letter2index, device = 'cpu'):
    gt_rep = np.zeros([len(word)+2], dtype=np.long)
    gt_rep[0]=0
    for letter_index, letter in enumerate(word):
        pos = letter2index[letter]
        gt_rep[letter_index+1] = pos
    gt_rep[letter_index+2]= letter2index[pad_char]
    return gt_rep
    word_rep('abc',eng_alpha2index).shape
    x=gt_rep('abc',eng_alpha2index)

In [8]:
gt_rep('abc',eng_alpha2index)

array([0, 2, 3, 4, 1])

In [9]:
def createDataset(dataset,letter2index):
    X_train=[]
    for i,data in enumerate(dataset):
        #print(data)
        X_train.append(gt_rep(data,letter2index))
    X_train = np.array(X_train)
    return X_train

In [10]:
X_train = createDataset(train_input_texts,eng_alpha2index)
y_train = createDataset(train_target_texts,hindi_alpha2index)
X_test = createDataset(test_input_texts,eng_alpha2index)
y_test = createDataset(test_target_texts,hindi_alpha2index)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [11]:
print(len(eng_alpha2index))
#a=[0]*(len(eng_alpha2index)-1)
a=[0]*128
eng_pad=[1]
eng_pad=eng_pad+a
print(eng_pad)
print(len(hindi_alpha2index))
a=[0]*128
hindi_pad=[1]
hindi_pad=hindi_pad+a
print(hindi_pad)

28
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
130
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [12]:
def one_hot_encode(sequence, n_unique):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)

In [13]:
one_hot_encode([1,2,3],10)

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]])

In [14]:
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]
'''d=one_hot_decode(y_train_padded[5])
c=hindi_index2alpha[6]+hindi_index2alpha[3]+hindi_index2alpha[22]+hindi_index2alpha[66]+hindi_index2alpha[49]+hindi_index2alpha[64]+hindi_index2alpha[37]+hindi_index2alpha[0]
c'''

'd=one_hot_decode(y_train_padded[5])\nc=hindi_index2alpha[6]+hindi_index2alpha[3]+hindi_index2alpha[22]+hindi_index2alpha[66]+hindi_index2alpha[49]+hindi_index2alpha[64]+hindi_index2alpha[37]+hindi_index2alpha[0]\nc'

In [15]:
max_input_sequence= max(len(seq) for seq in X_train)
max_output_sequence= max(len(seq) for seq in y_train)

print('max_input_sequence: ', max_input_sequence)
print('max_output_sequence: ', max_output_sequence)

max_input_sequence:  22
max_output_sequence:  21


In [16]:
from keras.preprocessing.sequence import pad_sequences
X_train_padded = pad_sequences(X_train, maxlen= max_input_sequence, padding='post', value=1)
print("X_train shape: ",X_train.shape)
print("X_train_padded shape: ",X_train_padded.shape)

y_train_padded = pad_sequences(y_train, maxlen= max_output_sequence, padding='post', value=1)
print("y_train shape: ",y_train.shape)
print("y_train_padded shape: ",y_train_padded.shape)

X_train shape:  (44204,)
X_train_padded shape:  (44204, 22)
y_train shape:  (44204,)
y_train_padded shape:  (44204, 21)


In [17]:
i=0
print("____Sample Input (Raw Format)____")
print("Original:\n", one_hot_decode(X_train[i]))
print("Padded:\n",one_hot_decode(X_train_padded[i]))
print("____Corresponding Output (Raw Format)____")
print("Original:\n", one_hot_decode(y_train[i]))
print("Padded:\n",one_hot_decode(y_train_padded[i]))

____Sample Input (Raw Format)____
Original:
 [0, 0, 0, 0]
Padded:
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
____Corresponding Output (Raw Format)____
Original:
 [0, 0, 0, 0]
Padded:
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [18]:
X_test_padded = pad_sequences(X_test, maxlen= max_input_sequence, padding='post', 
                              value=0)
print("X_test shape: ",X_test.shape)
print("X_test_padded shape: ",X_test_padded.shape)

y_test_padded = pad_sequences(y_test, maxlen= max_output_sequence, padding='post', 
                              value=0)
print("y_test shape: ",y_test.shape)
print("y_test_padded shape: ",y_test_padded.shape)

X_test shape:  (4502,)
X_test_padded shape:  (4502, 22)
y_test shape:  (4502,)
y_test_padded shape:  (4502, 21)


In [19]:
!pip3 install sklearn
from sklearn.model_selection import train_test_split
input_tensor, target_tensor = X_train_padded, y_train_padded
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.05)

In [20]:
# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [21]:
max_length_targ

21

In [22]:
target_tensor_train[0]

array([ 0, 23, 42, 79, 59, 74, 49, 64,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1], dtype=int32)

In [23]:
# Show the mapping b/w word index and language tokenizer
def convert(lang, tensor):
  for t in tensor:
    if t != 0 and t!=1:
      print ("%d ----> %s" % (t, lang[t]))
      
print ("Input Language; index to word mapping")
convert(eng_index2alpha, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(hindi_index2alpha, target_tensor_train[0])

Input Language; index to word mapping
12 ----> k
2 ----> a
15 ----> n
9 ----> h
2 ----> a
10 ----> i
26 ----> y
2 ----> a

Target Language; index to word mapping
23 ----> क
42 ----> न
79 ----> ्
59 ----> ह
74 ----> ै
49 ----> य
64 ----> ा


In [24]:
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

41993 41993 2211 2211


#End creating data set

In [25]:

def startTraining( batch_size=2048,embedding_dim=256,lstm_dim = 1024,attention_unit=35,EPOCHS=1,vocab_inp_size=len(eng_alpha2index) + 1,vocab_tar_size = len(hindi_alpha2index) + 1,optimizers="adam",attention_type="Bahdanau",lr=0.001):
    # Essential model parameters
    BUFFER_SIZE = len(input_tensor_train)
    BATCH_SIZE = batch_size
    steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
    embedding_dim = embedding_dim
    units = lstm_dim
    vocab_inp_size = vocab_inp_size
    vocab_tar_size = vocab_tar_size
    dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

    # Size of input and target batches
    example_input_batch, example_target_batch = next(iter(dataset))
    example_input_batch.shape, example_target_batch.shape

    # Encoder class
    class Encoder(tf.keras.Model):
        def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
            super(Encoder, self).__init__()
            self.batch_sz = batch_sz
            self.enc_units = enc_units

            # Embed the vocab to a dense embedding 
            self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

            # GRU Layer
            # glorot_uniform: Initializer for the recurrent_kernel weights matrix, 
            # used for the linear transformation of the recurrent state
            self.gru = CuDNNGRU(self.enc_units,
                                        return_sequences=True,
                                        return_state=True,
                                        recurrent_initializer='glorot_uniform')

        # Encoder network comprises an Embedding layer followed by a GRU layer
        def call(self, x, hidden):
            x = self.embedding(x)
            output, state = self.gru(x, initial_state=hidden)
            return output, state

        # To initialize the hidden state
        def initialize_hidden_state(self):
            return tf.zeros((self.batch_sz, self.enc_units))

    encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

    sample_hidden = encoder.initialize_hidden_state()
    sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)

    print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
    print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

    # Attention Mechanism
    class BahdanauAttention(tf.keras.layers.Layer):
        def __init__(self, units):
            super(BahdanauAttention, self).__init__()
            self.W1 = tf.keras.layers.Dense(units)
            self.W2 = tf.keras.layers.Dense(units)
            self.V = tf.keras.layers.Dense(1)

        def call(self, query, values):
            # query hidden state shape == (batch_size, hidden size)
            # values shape == (batch_size, max_len, hidden size)

            # we are doing this to broadcast addition along the time axis to calculate the score
            # query_with_time_axis shape == (batch_size, 1, hidden size)
            query_with_time_axis = tf.expand_dims(query, 1)

            # score shape == (batch_size, max_length, 1)
            # we get 1 at the last axis because we are applying score to self.V
            # the shape of the tensor before applying self.V is (batch_size, max_length, units)
            score = self.V(tf.nn.tanh(
                self.W1(query_with_time_axis) + self.W2(values)))

            # attention_weights shape == (batch_size, max_length, 1)
            attention_weights = tf.nn.softmax(score, axis=1)

            # context_vector shape after sum == (batch_size, hidden_size)
            context_vector = attention_weights * values
            context_vector = tf.reduce_sum(context_vector, axis=1)

            return context_vector, attention_weights
    


    class LuongAttention(tf.keras.layers.Layer):
        def __init__(self, units):
            super(LuongAttention, self).__init__()
            self.W1 = tf.keras.layers.Dense(units)
            self.W2 = tf.keras.layers.Dense(units)
            self.V = tf.keras.layers.Dense(1)

        def call(self, query, values):
            #print('\n******* Luong Attention  STARTS******')
            #print('query (decoder hidden state): (batch_size, hidden size) ', query.shape)
            #print('values (encoder all hidden state): (batch_size, max_len, hidden size) ', values.shape)

            # query hidden state shape == (batch_size, hidden size)
            # query_with_time_axis shape == (batch_size, 1, hidden size)
            # values shape == (batch_size, max_len, hidden size)
            # we are doing this to broadcast addition along the time axis to calculate the score
            query_with_time_axis = tf.expand_dims(query, 1)
            
            print('query_with_time_axis:(batch_size, 1, hidden size) ', query_with_time_axis.shape)


            values_transposed = tf.transpose(values, perm=[0, 2, 1])
            #print('values_transposed:(batch_size, hidden size, max_len) ', values_transposed.shape)

            # score shape == (batch_size, max_length, 1)
            # we get 1 at the last axis because we are applying score to self.V
            # the shape of the tensor before applying self.V is (batch_size, max_length, units)
            #BAHDANAU ADDITIVE:
            #score = self.V(tf.nn.tanh(
            #    self.W1(query_with_time_axis) + self.W2(values)))
            
            #LUONGH Dot-product
            score = tf.transpose(tf.matmul(query_with_time_axis, values_transposed) , perm=[0, 2, 1])

            print('score: (batch_size, max_length, 1) ',score.shape)
            # attention_weights shape == (batch_size, max_length, 1)
            attention_weights = tf.nn.softmax(score, axis=1)
            #print('attention_weights: (batch_size, max_length, 1) ',attention_weights.shape)
            # context_vector shape after sum == (batch_size, hidden_size)
            context_vector = attention_weights * values
            #print('context_vector before reduce_sum: (batch_size, max_length, hidden_size) ',context_vector.shape)
            context_vector = tf.reduce_sum(context_vector, axis=1)
            #print('context_vector after reduce_sum: (batch_size, hidden_size) ',context_vector.shape)


            #print('\n******* Luong Attention ENDS******')
            return context_vector, attention_weights
            
    attention_layer = BahdanauAttention(attention_unit)
    if attention_type=="Luong":
        attention_layer = LuongAttention(attention_unit)

    attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

    print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
    print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

    # Decoder class
    class Decoder(tf.keras.Model):
        def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
            super(Decoder, self).__init__()
            self.batch_sz = batch_sz
            self.dec_units = dec_units
            self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
            self.gru = CuDNNGRU(self.dec_units,
                                        return_sequences=True,
                                        return_state=True,
                                        recurrent_initializer='glorot_uniform')
            self.fc = tf.keras.layers.Dense(vocab_size)

            # Used for attention
            self.attention = BahdanauAttention(self.dec_units)

        def call(self, x, hidden, enc_output):
            # x shape == (batch_size, 1)
            # hidden shape == (batch_size, max_length)
            # enc_output shape == (batch_size, max_length, hidden_size)

            # context_vector shape == (batch_size, hidden_size)
            # attention_weights shape == (batch_size, max_length, 1)
            context_vector, attention_weights = self.attention(hidden, enc_output)

            # x shape after passing through embedding == (batch_size, 1, embedding_dim)
            x = self.embedding(x)

            # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
            x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

            # passing the concatenated vector to the GRU
            output, state = self.gru(x)

            # output shape == (batch_size * 1, hidden_size)
            output = tf.reshape(output, (-1, output.shape[2]))

            # output shape == (batch_size, vocab)
            x = self.fc(output)

            return x, state, attention_weights

    decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

    sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                        sample_hidden, sample_output)

    print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

    # Initialize optimizer and loss functions
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    if optimizers=="adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    if optimizers=="adagrad":
        optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)

    if optimizers=="sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr)

    if optimizers=="rmsprop":
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    if optimizers=="nadam":
        optimizer = tf.keras.optimizers.Nadam(learning_rate=lr)
        

    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')

    # Loss function
    def loss_function(real, pred):

        # Take care of the padding. Not all sequences are of equal length.
        # If there's a '0' in the sequence, the loss is being nullified
        mask = tf.math.logical_not(tf.math.equal(real, 0))
        loss_ = loss_object(real, pred)

        mask = tf.cast(mask, dtype=loss_.dtype)
        loss_ *= mask

        return tf.reduce_mean(loss_)


    def accuracy(targ, pred):
        #targ = K.argmax(y_train, axis=-1)
        pred = K.argmax(pred, axis=-1)
        #print(pred.shape)
        pred = K.cast(pred,"int32")
        #print(pred)
        #print("pred")
        # print(targ[:1].shape)
        #print("targ")
        #print(targ[:1])
        #predicted_id = tf.argmax(pred[0])
        #print(predicted_id)
        correct = K.cast(K.equal(targ, pred), dtype='float32')
        mask = K.cast(K.greater(targ, 0), dtype='float32') # filter out padding value 0.
        correctCount = K.sum(mask * correct)
        totalCount = K.sum(mask)
        return correctCount / totalCount

    import os

    checkpoint_dir = './training_checkpoints'
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
    checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                    encoder=encoder,
                                    decoder=decoder)

    @tf.function
    def train_step(inp, targ, enc_hidden):
        loss = 0
        acc=0

        # tf.GradientTape() -- record operations for automatic differentiation
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, enc_hidden)

            # dec_hidden is used by attention, hence is the same enc_hidden
            dec_hidden = enc_hidden

            # <start> token is the initial decoder input
            dec_input = tf.expand_dims([0] * BATCH_SIZE, 1)

            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):

                # Pass enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

                # Compute the loss
                loss += loss_function(targ[:, t], predictions)
                #print(predictions.shape,targ.shape)
                acc +=accuracy(targ[:, t], predictions)

                # Use teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)

        # As this function is called per batch, compute the batch_loss
        batch_loss = (loss / int(targ.shape[1]))
        batch_acc = (acc / int(targ.shape[1]))

        # Get the model's variables
        variables = encoder.trainable_variables + decoder.trainable_variables

        # Compute the gradients
        gradients = tape.gradient(loss, variables)

        # Update the variables of the model/network
        optimizer.apply_gradients(zip(gradients, variables))

        return batch_loss,batch_acc
    def split(word):
        return list(word)
        
    # Driver code
   # word = 'milan'
    #print(split(word))

    import numpy as np

    # Evaluate function -- similar to the training loop
    def evaluate(sentence):

        # Attention plot (to be plotted later on) -- initialized with max_lengths of both target and input
        attention_plot = np.zeros((max_length_targ, max_length_inp))

        '''# Preprocess the sentence given
        sentence = preprocess_sentence(sentence)

        # Fetch the indices concerning the words in the sentence and pad the sequence
        inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]'''
        inputs = gt_rep(sentence,eng_alpha2index)

        inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                                maxlen=max_length_inp,
                                                                padding='post')
        # Convert the inputs to tensors
        inputs = tf.convert_to_tensor(inputs)

        result = ''

        hidden = [tf.zeros((1, units))]
        enc_out, enc_hidden = encoder(inputs, hidden)

        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([0], 0)

        # Loop until the max_length is reached for the target lang (ENGLISH)
        for t in range(max_length_targ):
            predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                                dec_hidden,
                                                                enc_out)
            #print(predictions.shape)

            # Store the attention weights to plot later on
            attention_weights = tf.reshape(attention_weights, (-1, ))
            attention_plot[t] = attention_weights.numpy()

            # Get the prediction with the maximum attention
            predicted_id = tf.argmax(predictions[0]).numpy()

            # Append the token to the result
            if hindi_index2alpha[predicted_id]!='<end>':
                result += hindi_index2alpha[predicted_id]

            # If <end> token is reached, return the result, input, and attention plot
            if hindi_index2alpha[predicted_id] == '<end>':
                return result, sentence, attention_plot

            # The predicted ID is fed back into the model
            dec_input = tf.expand_dims([predicted_id], 0)

        return result, sentence, attention_plot

    from matplotlib.font_manager import FontProperties
    from pathlib import Path
    '''nirm = Path('/content/Nirmala.ttf')
    # configure the Hindi font
    hindi_font = FontProperties(fname=nirm)'''

    import matplotlib.pyplot as plt
    from matplotlib.pyplot import figure
    from matplotlib.font_manager import FontProperties


    hindi_font = FontProperties(fname='Mangal 400.TTF', size=18)


    import matplotlib.pyplot as plt
    import matplotlib.ticker as ticker

    # Function for plotting the attention weights
    def plot_attention(attention, sentence, predicted_sentence):
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(1, 1, 1)
        ax.matshow(attention, cmap='viridis')

        fontdict = {'fontsize': 14}

        ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
        ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
        for label in  ax.get_yticklabels():
            label.set_fontproperties(hindi_font)
            label.set_fontsize(15) 

        plt.show()

    # Translate function (which internally calls the evaluate function)
    def translate2(sentence):
        result, sentence, attention_plot = evaluate(sentence)

        print('Input: %s' % (sentence))
        print('Predicted translation: {}'.format(result))

        attention_plot = attention_plot[:len(result), :len(sentence)]
        plot_attention(attention_plot, split(sentence), split(result))
        return result

    # Translate function (which internally calls the evaluate function)
    def translate(sentence):
        result, sentence, attention_plot = evaluate(sentence)

        # print('Input: %s' % (sentence))
        #print('Predicted translation: {}'.format(result))

        #attention_plot = attention_plot[:len(result), :len(sentence)]
        # plot_attention(attention_plot, split(sentence), split(result))
        return result












    def calculate_Sample_Word_accuracy(val_input,val_target,sample_size):
        randomlist = random.sample(range(0, len(val_input)), sample_size)
        # print(randomlist)
        count=0
        start = time.time()
        for i in randomlist:
            result = translate(val_input[i])
            #print("Actual translation",val_target[i])
            if(result==val_target[i]):
                count+=1
        #print('Val Accuracy = : {}'.format(count*100.0/len(randomlist)))
        #print('Time taken for calculating word accuracy {} sec\n'.format(time.time() - start))
        #print("\n")
        #print("\n")
        return count*100.0/len(randomlist)

    import time



    # Training loop
    for epoch in range(EPOCHS):
        start = time.time()

        # Initialize the hidden state
        enc_hidden = encoder.initialize_hidden_state()
        total_loss = 0
        total_acc=0
        # Loop through the dataset
        for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):

            # Call the train method
            batch_loss,batch_acc = train_step(inp, targ, enc_hidden)

            # Compute the loss (per batch)
            total_loss += batch_loss
            total_acc += batch_acc

            if batch % 100 == 0:
                print('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1,
                                                            batch,
                                                            batch_loss.numpy(),batch_acc.numpy()))
        # Save (checkpoint) the model every 2 epochs
        if (epoch + 1) % 2 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

        # Output the loss observed until that epoch
        print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                            total_loss / steps_per_epoch))
        wandb.log({"Train Loss": total_loss / steps_per_epoch})
        # Output the loss observed until that epoch
        print('Epoch {} Train Character_level Accuracy {:.4f}'.format(epoch + 1,
                                            total_acc / steps_per_epoch))
        wandb.log({"Train Accuracy": total_acc / steps_per_epoch})


        train_word_accuracy = calculate_Sample_Word_accuracy(train_input_texts,train_target_texts,150)
        val_accuracy = calculate_Sample_Word_accuracy(val_input_texts,val_target_texts,150)
        print('Epoch {} Train_Samplw_Word Accuracy {:.4f}'.format(epoch + 1,
                                      train_word_accuracy))
        wandb.log({"Train_Samplw_Word Accuracy": train_word_accuracy})
        print('Epoch {} Validation_Sample_Word_Accuracy {:.4f}'.format(epoch + 1,
                                      val_accuracy))
        wandb.log({"Validn_Samplw_Word Accuracy": val_accuracy})


        
        print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))


   

    #translate2("sentence")

    # Restore the latest checkpoint in checkpoint_dir
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

    #result = translate("krishna")
    #result

    def calculate_wordAccuracy(input_text,target_text):
        start = time.time()
        count=0
        for (i,sentence) in enumerate(input_text):
            #print(sentence)
            result = translate(sentence)
            #print("Actual translation",target_text[i])
            if(result==target_text[i]):
                count+=1
        print('Test Sample Word Accuracy = : {}'.format(count*100.0/len(input_text)))
        return count*100.0/len(input_text)


    test_accuracy = calculate_wordAccuracy(test_input_texts[0:1000],test_target_texts[0:1000])
    wandb.log({"Test_Sample_word_accuracy": test_accuracy})






In [26]:
sweep_config = {
    'name': 'Hyper_parameter_check',
    'method': 'random', #grid, random, bayes
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters':{
        'batch_size':{
           'values' :[128,256,512]
        },
        'max_epoch':{
            'values':[10,15,20,30]
        },
        
        'embedding_dim':{
            'values':[64,256,512]
        },
        'enc_dim':{
            'values': [256,512,1024]
        },
        'vocab_inp_size':{
            'values': [27,50,100]
        },
        'vocab_tar_size':{
            'values': [140,200,250]
        },
        'attention_unit':{
            'values': [25,35,50]
        },
        'attention_type':{
            'values': ["Luong","Bahdanau"]
        },
        'optimizers':{
            'values': ["adam","rmsp"]
        },
        'EPOCHS':{
            'values': [15,20]
        },
        'lr':{
            'values': [0.001,0.0001,0.01]
        }

    },
}

In [27]:
wandb.login()
wandb.init(project = "Assignment 3 Part 2", entity = "cs20m038")
sweep_id = wandb.sweep(sweep_config, project='Assignment 3 Part 2', entity='cs20m038')

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs20m038 (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 38t7i5m2
Sweep URL: https://wandb.ai/cs20m038/Assignment%203%20Part%202/sweeps/38t7i5m2


In [28]:
def runSweep():
    run = wandb.init()
    configuration = run.config
    startTraining( batch_size=configuration.batch_size,embedding_dim=configuration.embedding_dim,lstm_dim = configuration.enc_dim,attention_unit=configuration.attention_unit,EPOCHS=configuration.EPOCHS,vocab_inp_size=configuration.vocab_inp_size,vocab_tar_size = configuration.vocab_tar_size,optimizers=configuration.optimizers,attention_type=configuration.attention_type,lr=configuration.lr)
    #startTraining(batch_size=configuration.batch_size,embedding_dim=configuration.embedding_dim,lstm_dim = configuration.enc_dim ,attention_unit=configuration.attention_unit,EPOCHS=configuration.EPOCHS,vocab_inp_size=configuration.vocab_inp_size,vocab_tar_size = configuration.vocab_tar_size,optimizers=configuration.optimizers,attention_type=configuration.attention_type,lr=configuration.lr)
    #startTraining(model_name = configuration.model_name, num_classess = 10, batch_size = configuration.batch_size, num_epochs=configuration.max_epoch,feature_extract = False,no_layers_to_unfreeze=configuration.layers_to_train,lr=configuration.learning_rate,gamma=configuration.gamma,opt = configuration.optimizer)
   # startTraining( batch_size=configuration.batch_size,embedding_dim=configuration.embedding_dim,lstm_units = 1024,attention_unit=35,EPOCHS=1,vocab_inp_size=len(eng_alpha2index) + 1,vocab_tar_size = len(hindi_alpha2index) + 1,optimizers="adam",attention_type="Bahdanau",lr=0.001)

In [ ]:
wandb.agent(sweep_id=sweep_id, function=runSweep)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: ns6dcil1 with config:
wandb: 	EPOCHS: 20
wandb: 	attention_type: Luong
wandb: 	attention_unit: 50
wandb: 	batch_size: 128
wandb: 	embedding_dim: 64
wandb: 	enc_dim: 512
wandb: 	lr: 0.0001
wandb: 	max_epoch: 15
wandb: 	optimizers: rmsp
wandb: 	vocab_inp_size: 27
wandb: 	vocab_tar_size: 200


wandb: ERROR Run ns6dcil1 errored: InvalidArgumentError()
wandb: Agent Starting Run: lioi1q0u with config:
wandb: 	EPOCHS: 15
wandb: 	attention_type: Bahdanau
wandb: 	attention_unit: 50
wandb: 	batch_size: 512
wandb: 	embedding_dim: 512
wandb: 	enc_dim: 1024
wandb: 	lr: 0.01
wandb: 	max_epoch: 10
wandb: 	optimizers: rmsp
wandb: 	vocab_inp_size: 27
wandb: 	vocab_tar_size: 250


wandb: ERROR Run lioi1q0u errored: InvalidArgumentError()
wandb: Agent Starting Run: gmb13mr5 with config:
wandb: 	EPOCHS: 15
wandb: 	attention_type: Luong
wandb: 	attention_unit: 35
wandb: 	batch_size: 128
wandb: 	embedding_dim: 256
wandb: 	enc_dim: 512
wandb: 	lr: 0.0001
wandb: 	max_epoch: 20
wandb: 	optimizers: adam
wandb: 	vocab_inp_size: 27
wandb: 	vocab_tar_size: 250


Encoder output shape: (batch size, sequence length, units) (128, 22, 512)
Encoder Hidden state shape: (batch size, units) (128, 512)
query_with_time_axis:(batch_size, 1, hidden size)  (128, 1, 512)
score: (batch_size, max_length, 1)  (128, 22, 1)
Attention result shape: (batch size, units) (128, 512)
Attention weights shape: (batch_size, sequence_length, 1) (128, 22, 1)
Decoder output shape: (batch_size, vocab size) (128, 250)


wandb: ERROR Run gmb13mr5 errored: InvalidArgumentError()
wandb: Agent Starting Run: tfrbjzm7 with config:
wandb: 	EPOCHS: 15
wandb: 	attention_type: Bahdanau
wandb: 	attention_unit: 50
wandb: 	batch_size: 512
wandb: 	embedding_dim: 512
wandb: 	enc_dim: 512
wandb: 	lr: 0.0001
wandb: 	max_epoch: 10
wandb: 	optimizers: adam
wandb: 	vocab_inp_size: 100
wandb: 	vocab_tar_size: 200


...

Encoder output shape: (batch size, sequence length, units) (256, 22, 1024)
Encoder Hidden state shape: (batch size, units) (256, 1024)
query_with_time_axis:(batch_size, 1, hidden size)  (256, 1, 1024)
score: (batch_size, max_length, 1)  (256, 22, 1)
Attention result shape: (batch size, units) (256, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (256, 22, 1)
Decoder output shape: (batch_size, vocab size) (256, 200)
Epoch 1 Batch 0 Loss 5.0452 Accuracy 0.0009
Epoch 1 Batch 100 Loss 1.1742 Accuracy 0.6564
Epoch 1 Loss 1.4646
Epoch 1 Train Character_level Accuracy 0.6472
Epoch 1 Train_Samplw_Word Accuracy 0.0000
Epoch 1 Validation_Sample_Word_Accuracy 0.0000
Time taken for 1 epoch 78.72328019142151 sec

Epoch 2 Batch 0 Loss 1.1331 Accuracy 0.6642
Epoch 2 Batch 100 Loss 1.0653 Accuracy 0.6743
Epoch 2 Loss 1.0910
Epoch 2 Train Character_level Accuracy 0.6672
Epoch 2 Train_Samplw_Word Accuracy 0.0000
Epoch 2 Validation_Sample_Word_Accuracy 0.0000
Time taken for 1 epoch 67.695

Train Loss,0.36735
_runtime,1667
_timestamp,1621932222
_step,60
Train Accuracy,0.83643
Train_Samplw_Word Accuracy,0.0
Validn_Samplw_Word Accuracy,0.0
Test_Sample_word_accuracy,0.0


Train Loss,█▆▅▅▄▄▃▃▂▂▁▁▁▁▁
_runtime,▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇█
_timestamp,▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇█
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Train Accuracy,▁▂▂▃▃▄▅▅▆▇▇████
Train_Samplw_Word Accuracy,▁▁▁▁█▁▁▁▁▁▁▁▁▁▁
Validn_Samplw_Word Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_Sample_word_accuracy,▁


wandb: Agent Starting Run: vjfi4082 with config:
wandb: 	EPOCHS: 20
wandb: 	attention_type: Bahdanau
wandb: 	attention_unit: 50
wandb: 	batch_size: 256
wandb: 	embedding_dim: 256
wandb: 	enc_dim: 256
wandb: 	lr: 0.001
wandb: 	max_epoch: 10
wandb: 	optimizers: adam
wandb: 	vocab_inp_size: 50
wandb: 	vocab_tar_size: 250


Encoder output shape: (batch size, sequence length, units) (256, 22, 256)
Encoder Hidden state shape: (batch size, units) (256, 256)
Attention result shape: (batch size, units) (256, 256)
Attention weights shape: (batch_size, sequence_length, 1) (256, 22, 1)
Decoder output shape: (batch_size, vocab size) (256, 250)
Epoch 1 Batch 0 Loss 5.2546 Accuracy 0.0002
Epoch 1 Batch 100 Loss 1.0746 Accuracy 0.6780
Epoch 1 Loss 1.4044
Epoch 1 Train Character_level Accuracy 0.6593
Epoch 1 Train_Samplw_Word Accuracy 0.0000
Epoch 1 Validation_Sample_Word_Accuracy 0.0000
Time taken for 1 epoch 80.82644295692444 sec

Epoch 2 Batch 0 Loss 1.0304 Accuracy 0.6752
Epoch 2 Batch 100 Loss 0.9573 Accuracy 0.6981
Epoch 2 Loss 0.9628
Epoch 2 Train Character_level Accuracy 0.6942
Epoch 2 Train_Samplw_Word Accuracy 0.0000
Epoch 2 Validation_Sample_Word_Accuracy 0.0000
Time taken for 1 epoch 66.73927068710327 sec

Epoch 3 Batch 0 Loss 0.9217 Accuracy 0.7037
Epoch 3 Batch 100 Loss 0.8794 Accuracy 0.7117
Epoch 3 Los